In [1]:
import json
import uuid
import requests

## Test embedding, context, chat completion

##### Test chat_completion

In [5]:
# Make chat_completion call
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "system_message": "You are a helpful assistant that translates English to French.",
    "user_message": "Translate this sentence from English to French. I love programming.",
}

url = "http://localhost:8000/chat_completion"
r = requests.post(url, json=payload)
r.raise_for_status()
r.json()

{'id': '008f2686361c48b9a3ff495feaa9628c'}

In [6]:
# retrieve the response from the server

get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r = r.json(); r

[{'id': '008f2686361c48b9a3ff495feaa9628c',
  'system_message': 'You are a helpful assistant that translates English to French.',
  'user_message': 'Translate this sentence from English to French. I love programming.',
  'response': "J'adore la programmation.",
  'request_type': 'chat_completion',
  'request_time': '1695380308.6723156',
  'response_time': '1695380313.454599',
  'processed': True,
  'processor': '0x9965507D1a55bcC2695C58ba16FB37d819B0A4dc',
  'error': False,
  'error_message': None,
  'error_name': None}]

In [7]:
# Make multiple calls

ids = []
system_messages = [
    "You are a helpful assistant that translates English to French.",
    "You are a helpful assistant that translates English to Italian.",
    "You are a helpful assistant that translates English to Spanish.",
    "You are a helpful assistant that translates English to German.",
    "You are a helpful assistant that translates English to Russian.",
]

user_messages = [
    "Translate this sentence from English to French. I love programming.",
    "Translate this sentence from English to Italian. I love programming.",
    "Translate this sentence from English to Spanish. I love programming.",
    "Translate this sentence from English to German. I love programming.",
    "Translate this sentence from English to Russian. I love programming.",
]
request_responses = []
for i in range(5):
    id_ = uuid.uuid4().hex
    payload = {
        "id": id_,
        "system_message": system_messages[i],
        "user_message": user_messages[i],
    }
    ids.append(id_)
    url = "http://localhost:8000/chat_completion"
    r = requests.post(url, json=payload)
    request_responses.append(r)

In [9]:
# retrieve multiple responses from the server

for id_, r in zip(ids, request_responses):
    get_url = f"http://localhost:8000/get/{id_}"
    r = requests.get(get_url)
    print(r.json())

[{'id': 'c198194da15543bbb381afece44b540f', 'system_message': 'You are a helpful assistant that translates English to French.', 'user_message': 'Translate this sentence from English to French. I love programming.', 'response': "J'adore la programmation.", 'request_type': 'chat_completion', 'request_time': '1695380342.8548272', 'response_time': '1695380347.4400806', 'processed': True, 'processor': '0x9965507D1a55bcC2695C58ba16FB37d819B0A4dc', 'error': False, 'error_message': None, 'error_name': None}]
[{'id': '758ea3c5e40c4a36bcc483a874697cb4', 'system_message': 'You are a helpful assistant that translates English to Italian.', 'user_message': 'Translate this sentence from English to Italian. I love programming.', 'response': 'Amo la programmazione.', 'request_type': 'chat_completion', 'request_time': '1695380342.8641956', 'response_time': '1695380347.4515853', 'processed': True, 'processor': '0x976EA74026E726554dB657fA54763abd0C3a0aa9', 'error': False, 'error_message': None, 'error_nam

##### Embedding

In [10]:
# Upload a file to create an embedding.
# NOTE: For pdf files, you need to parse locally and upload the text file.

emb_id_ = uuid.uuid4().hex
payload = {
    "id": emb_id_,
    # "system_message": "You are a helpful assistant that translates English to French.",
    # "user_message": "Translate this sentence from English to French. I love programming.",
}

# Add a file
file = open("../../README.md", "rb")
files = {"file": file}

url = "http://localhost:8000/embedding"
r = requests.post(url, files=files, data={"data":json.dumps(payload)})
r.raise_for_status()
r.json()

{'id': '191776a64afe4af48d4696ccea3a3eec'}

##### chat

In [11]:
# Chatting over the embedding above. Since no `memory_id` is provided, the agent will assume this is a new conversation
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": emb_id_, # Is the same as the id when making the embedding
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
r = r.json()
mem_id = r["memory_id"]


In [37]:
get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r.raise_for_status()
r = r.json(); r

[{'id': '10467ca444a24624ba32ae9fe156d81b',
  'context_id': 'b19b850793774c8785c5ee916946b778',
  'memory_id': '5d89536018fc4968b9b61c0cb6cd56ed',
  'question': 'What is daios?',
  'modified_question': None,
  'response': 'Based on the context provided, DAIOS refers to the Decentralized AI Operating System. It is an extension of the Open AEA (Agent Environment Architecture) framework developed by Valory. The DAIOS framework aims to build a decentralized operating system for artificial intelligence, integrating blockchain, messaging, and other technologies.',
  'chat_history': [{'role': 'user', 'content': 'What is daios?'},
   {'role': 'assistant',
    'content': 'Based on the context provided, DAIOS refers to the Decentralized AI Operating System. It is an extension of the Open AEA (Agent Environment Architecture) framework developed by Valory. The DAIOS framework aims to build a decentralized operating system for artificial intelligence, integrating blockchain, messaging, and other te

In [12]:
# since a `memory_id` is provided, the agent will use the memory to generate a follow-up response
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": emb_id_,
    "memory_id": mem_id,
    "question": "does it provide a platform?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
r.json()

{'id': '098b704a979e4ddfbe1f719ded51374b',
 'memory_id': '46f45ef144524af095ecfddb30ef814e'}

In [13]:
get_url = f"http://localhost:8000/get/{id_}"

r = requests.get(get_url)
r = r.json()
r

[{'id': '098b704a979e4ddfbe1f719ded51374b',
  'context_id': '191776a64afe4af48d4696ccea3a3eec',
  'memory_id': '46f45ef144524af095ecfddb30ef814e',
  'question': 'does it provide a platform?',
  'modified_question': 'Does DAIOS provide a platform?',
  'response': 'Based on the context provided, DAIOS does not explicitly mention whether it provides a platform. However, it is described as a decentralized operating system for AI applications. Therefore, it can be inferred that DAIOS may provide a platform for building and running AI applications.',
  'chat_history': [{'role': 'user', 'content': 'What is daios?'},
   {'role': 'assistant',
    'content': 'Based on the context provided, DAIOS stands for Decentralized AI Operating System. It is an extension of the Open Autonomy framework, which is a framework for building autonomous agents in the crypto space. DAIOS aims to provide a decentralized operating system for AI applications.'},
   {'role': 'user', 'content': 'Does DAIOS provide a pla

##### Bad requests are handled within the http_server

In [40]:
# non-existent memory
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": "bf68b34c19964a73ba03c9779ef70fa9",
    "memory_id": "87b9426ab1f84ad689e0eb29e350db83",
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
print(r.status_code)


HTTPError: 400 Client Error: context_id bf68b34c19964a73ba03c9779ef70fa9 does not exist for url: http://localhost:8000/chat

In [13]:
# non-existent context
id_ = uuid.uuid4().hex
payload = {
    "id": id_,
    "context_id": "bf68b34c19964a73ba03c9779ef70fa9",
    "question": "What is daios?",
}

url = "http://localhost:8000/chat"
r = requests.post(url, json=payload)
r.raise_for_status()
print(r.status_code)


HTTPError: 400 Client Error: context_id bf68b34c19964a73ba03c9779ef70fa9 does not exist for url: http://localhost:8000/chat

### V2 (Deprecated)

In [1]:
import pika
import json
import uuid

In [ ]:
credentials = pika.PlainCredentials('guest', 'guest')
parameters = pika.ConnectionParameters('localhost', 5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
# channel.queue_declare(queue="consume_queue")


id = uuid.uuid4().hex

# basic publish
channel.basic_publish(
    exchange='',
    routing_key='consume_queue',
    body=json.dumps({"id":id, "user_message": "What is the capital of India?"})
)

id

In [ ]:
credentials = pika.PlainCredentials('guest', 'guest')
parameters = pika.ConnectionParameters('localhost', 5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
# channel.queue_declare(queue="consume_queue")

# basic publish
method_frame, header_frame, body = channel.basic_get(queue="publish_queue", auto_ack=True)

body

### Deprecated

In [2]:
rmq = InteractRabbitMQ()
rmq.start()
id_ = uuid.uuid4().hex
req = {"request": "What is the capital of Indonesia", "id": id_}

In [3]:
rmq.enqueue_request(req)

In [4]:
res = rmq.get_response_queue(id_)

In [5]:
res

{'id': '3856e399bb064584a2a650c73d1f7b2c',
 'request': 'What is the capital of Indonesia',
 'response': 'The capital of Indonesia is Jakarta.'}